In [ ]:
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()

In [1]:
import os 
import sys 
import json 
import random 
import logging 
import nltk 
import tensorflow as tf 
from tensorflow.keras.utils import Progbar 
from glob import glob 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [2]:
from bert_package import modeling, optimization, tokenization 
from bert_package.run_pretraining import input_fn_builder, model_fn_builder

In [3]:
import csv

bert_corpus = []
df = spark.read.csv("hdfs:///user/spark/datafile/csi_pos_neg.csv", header=True)
df = df.toPandas()
for line in df['text'][:99756]:
    bert_corpus.append(str(line.strip()))
bert_corpus

/root/spark/python/pyspark/sql/dataframe.py:2111: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  PyArrow >= 0.8.0 must be installed; however, it was not found.
Attempting non-optimization as 'spark.sql.execution.arrow.fallback.enabled' is set to true.
  warnings.warn(msg)


['threatmeter hacked emails of san francisco muni rail system hacker reveals clues about identity and tactics kr  fin',
 'first android malware targeting pcs uncovered  fbxtynm',
 'adobe fixes six code execution bugs in flash  mokgxr',
 'scienceporn  in a vacuum i guess',
 'riskware hmoyfzb  fbjaz',
 'rt exploithunt  wzemae   data protection in fusion itsec  databreach  news   wzenei',
 'webroot  spamvertised you have  lost message on facebook campaign leads to pharmaceutical scams  xunah',
 'cybersecurity  hacker  privacy and big data   rmge',
 'postsnag gingcredsfromlockedmachines',
 'ddos  t  mvrzcc',
 'top  actions for runtime application protection  fbodttge',
 'watch  aqvqvy',
 'threatmeter the facebook algorithm is watching you  loetua',
 'product model v product name forensic ultradock v product ty  rltxbn   cybersecurity  bitcoin',
 'rt securitytube  video ettercap beef mashup',
 'you never know when a moment and a few sincere words can have an impact on a life thezigziglar  s

In [4]:
with open("./bert_corpus.txt", 'w') as output:
    for row in bert_corpus:
        output.write(str(row) + '\n')

In [5]:
check_data = [] 
with open('./bert_corpus.txt', 'r') as f: 
    for i in range(10): 
        check_data.append(f.readline()) 
check_data

['threatmeter hacked emails of san francisco muni rail system hacker reveals clues about identity and tactics kr  fin\n',
 'first android malware targeting pcs uncovered  fbxtynm\n',
 'adobe fixes six code execution bugs in flash  mokgxr\n',
 'scienceporn  in a vacuum i guess\n',
 'riskware hmoyfzb  fbjaz\n',
 'rt exploithunt  wzemae   data protection in fusion itsec  databreach  news   wzenei\n',
 'webroot  spamvertised you have  lost message on facebook campaign leads to pharmaceutical scams  xunah\n',
 'cybersecurity  hacker  privacy and big data   rmge\n',
 'postsnag gingcredsfromlockedmachines\n',
 'ddos  t  mvrzcc\n']

In [6]:
from tokenizers import BertWordPieceTokenizer 
tokenizer = BertWordPieceTokenizer(lowercase=False, strip_accents=False) 
tokenizer.train(['./bert_corpus.txt'], vocab_size=13004, limit_alphabet=3000)

In [7]:
tokenizer.save_model('.')

['./vocab.txt']

In [8]:
!tail -n 20 vocab.txt

amazonmetadescription
vempraruabr
ao
ae
boun
bub
bbb
bird
cort
ccc
crap
dob
dance
dsl
eine
esa
fer
gender
gemal
ggers


In [9]:
MAX_SEQ_LENGTH = 128 #@param {type:"integer"} 
MASKED_LM_PROB = 0.15 #@param 
MAX_PREDICTIONS = 20#@param {type:"integer"} 
DO_LOWER_CASE = False #@param {type:"boolean"} 
PROCESSES = 4 #@param {type:"integer"} 
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"} 
VOC_FNAME = 'vocab.txt' #@param {type:"string"}

In [24]:
XARGS_CMD = ("ls ./shards/ | " 
             "xargs -n 1 -P {} -I{} " 
             "python3 bert_package/create_pretraining_data.py " 
             "--input_file=./shards/{} " 
             "--output_file={}/{}.tfrecord " 
             "--vocab_file={} " 
             "--do_lower_case={} " 
             "--max_predictions_per_seq={} " 
             "--max_seq_length={} " 
             "--masked_lm_prob={} " 
             "--random_seed=34 " 
             "--dupe_factor=5") 

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                             VOC_FNAME, DO_LOWER_CASE, 
                             MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB) 

tf.io.gfile.mkdir(PRETRAINING_DIR) 

In [25]:
!$XARGS_CMD

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

INFO:tensorflow:*** Writing to output files ***
INFO:tensorflow:  pretraining_data/shard_0001.tfrecord
INFO:tensorflow:*** Example ***
INFO:tensorflow:tokens: [CLS] than [MASK] ##tw ##e ##jk ##n securityaffairs android boot ##kit malware opdevsec rt qu ##tt ##era qu ##tt ##era wordpress malware scanner has been released added new malware samples [MASK] [MASK] ##mal [MASK] are [MASK] [MASK] how big [MASK] can help you choose better health insurance fast [MASK] ##com ##how ##b ig ##data ##can ##help ##you ##ch ##oo [MASK] ##bet ##ter ##health ##ins [SEP] media [MASK] ##ma ##ver o ##eff ##ent ##lich ##ung ##enc ##oop ##oa ##kl ##and ##pdf cybersecurity hacker build your [MASK] security [MASK] a field [MASK] for network testing tobemcomcyberwar ##build your ##own ##security ##lab [MASK] [MASK] ##guide ##for ##network ##testing cyberwarbooks if your job is to design or implement it charles [MASK] or if youre [MASK] ##ing for any security certification t [SEP]
INFO:tensorflow:input_ids: 2 50

INFO:tensorflow:*** Example ***
INFO:tensorflow:tokens: [CLS] gu ##te lo ##bb ##ya ##rb ##ei [MASK] le ##ist ##en wie [MASK] ##i der cloud und die sch ##afe [MASK] ##gen when [MASK] ta ##ste [MASK] excess everything else ta ##ste ##s bl ##and six ##x am theathelite quote rdp ##arty confident behind of healthcare databreach ##es itsecuritynewsinfor ##dparty ##ven ##d [MASK] ##sb ##eh ##ind ##of ##health ##care ##databreach ##es bab ##ar and cas ##per [MASK] [MASK] likely designed [SEP] ct ##vn ##ews kids electronics ##maker vt ##ech says m customer accounts compromised in database [MASK] ct ibd ##d wu ##wb ##i ic ##ym ##i the fbi is [MASK] ##ing people into being worried [MASK] weird impl ##aus ##ible the ##ories of terrorism thegrugq ##fe ##eb [MASK] ##no [MASK] ##pol [MASK] ##ti ##ona ##cb ##a [SEP]
INFO:tensorflow:input_ids: 2 427 989 1082 1280 5951 6678 2055 4 199 193 68 12301 4 43 3014 584 1416 3050 941 3535 4 3043 669 4 6061 3500 4 11453 2058 4646 6061 3500 44 465 172 3943 53 445 

In [12]:
#BUCKET_NAME = "csiposneg" #@param {type:"string"} 
MODEL_DIR = "bert_model" #@param {type:"string"} 
tf.io.gfile.mkdir(MODEL_DIR) 
assert BUCKET_NAME != ''

In [13]:
# Token size, Max Length를 제외한 나머지는 Bert Base 모델과 동일한 파라미터 
bert_base_config = { "attention_probs_dropout_prob": 0.1, 
                    "directionality": "bidi", 
                    "hidden_act": "gelu", 
                    "hidden_dropout_prob": 0.1, 
                    "hidden_size": 768, 
                    "initializer_range": 0.02, 
                    "intermediate_size": 3072,
                    "max_position_embeddings": 512, 
                    "num_attention_heads": 12, 
                    "num_hidden_layers": 12, 
                    "pooler_fc_size": 768, 
                    "pooler_num_attention_heads": 12, 
                    "pooler_num_fc_layers": 3, 
                    "pooler_size_per_head": 128, 
                    "pooler_type": "first_token_transform", 
                    "type_vocab_size": 2, 
                    "vocab_size": 13004}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo: 
    json.dump(bert_base_config, fo, indent=2) 

In [14]:
!cp vocab.txt $MODEL_DIR

In [15]:
!ls $MODEL_DIR

bert_config.json  vocab.txt


In [30]:
# Input data pipeline config 
TRAIN_BATCH_SIZE = 128 #@param {type:"integer"} 
MAX_PREDICTIONS = 20 #@param {type:"integer"} 
MAX_SEQ_LENGTH = 128 #@param {type:"integer"} 
MASKED_LM_PROB = 0.15 #@param 

# Training procedure config 
EVAL_BATCH_SIZE = 64 
LEARNING_RATE = 2e-5 
TRAIN_STEPS = 1000000 #@param {type:"integer"} 
SAVE_CHECKPOINTS_STEPS = 2500 #@param {type:"integer"} 
NUM_TPU_CORES = 8 

BERT_GCS_DIR = "./bert_model"
DATA_GCS_DIR = "./pretraining_data"

VOCAB_FILE = os.path.join(BERT_GCS_DIR, VOC_FNAME) 
CONFIG_FILE = os.path.join(BERT_GCS_DIR, "bert_config.json") 

INIT_CHECKPOINT = tf.train.latest_checkpoint(BERT_GCS_DIR) 

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE) 
input_files = tf.io.gfile.glob(os.path.join(DATA_GCS_DIR,'*tfrecord')) 

In [31]:
USE_TPU = False
model_fn = model_fn_builder( bert_config=bert_config, init_checkpoint=INIT_CHECKPOINT, learning_rate=LEARNING_RATE, num_train_steps=TRAIN_STEPS, num_warmup_steps=10000, use_tpu=USE_TPU, use_one_hot_embeddings=True)

In [32]:
run_config = tf.contrib.tpu.RunConfig(
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

INFO:tensorflow:Using config: {'_model_dir': './bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa91d05d5c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2500, num_shards=8, num_cores_per_replica=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None, input_

In [ ]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running train on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (128, 128)
INFO:tensorflow:  name = input_mask, shape = (128, 128)
INFO:tensorflow:  name = masked_lm_ids, shape = (128, 20)
INFO:tensorflow:  name = masked_lm_positions, shape = (128, 20)
INFO:tensorflow:  name = masked_lm_weights, shape = (128, 20)
INFO:tensorflow:  name = next_sentence_labels, shape = (128, 1)
INFO:tensorflow:  name = segment_ids, shape = (128, 128)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (13004, 768)
INFO:tensorflow:  name = bert/embeddings/token_type_embed

INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072)
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,)
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768)
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/bias:0, sha

INFO:tensorflow:  name = bert/encoder/layer_9/output/dense/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/beta:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_9/output/LayerNorm/gamma:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/bias:0, shape = (768,)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768)
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/bias:0, shape = 

# Tensorflow2 Implementation, But failed because of dependency issue

In [20]:
mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0","/gpu:1","/gpu:2","/gpu:3"],
                      cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())

AttributeError: module 'tensorflow._api.v1.distribute' has no attribute 'HierarchicalCopyAllReduce'

In [21]:
tf.compat.v1.disable_eager_execution()

In [20]:
with mirrored_strategy.scope():
    model_fn = model_fn_builder(
        bert_config=bert_config,
        init_checkpoint=INIT_CHECKPOINT,
        learning_rate=LEARNING_RATE,
        num_train_steps=TRAIN_STEPS,
        num_warmup_steps=10,
        use_tpu=False,
        use_one_hot_embeddings=True)

In [21]:
multi_run_config = tf.compat.v1.estimator.RunConfig(
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [35]:
estimator = tf.compat.v1.estimator.Estimator(
    model_fn = model_fn,
    config = multi_run_config,
    params = {"batch_size":128})

INFO:tensorflow:Using config: {'_model_dir': './bert_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [36]:
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

In [27]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)

INFO:tensorflow:Error recorded from training_loop: module 'tensorflow.compat.v1' has no attribute 'contrib'
INFO:tensorflow:training_loop marked as finished


AttributeError: module 'tensorflow.compat.v1' has no attribute 'contrib'

In [ ]:
model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=LEARNING_RATE,
      num_train_steps=TRAIN_STEPS,
      num_warmup_steps=10,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=True)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=BERT_GCS_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=SAVE_CHECKPOINTS_STEPS,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS,
        is_training=True)

In [ ]:
estimator.train(input_fn=train_input_fn, max_steps=TRAIN_STEPS)